In [1]:
from setup import *
from scipy.special import binom

srt = 'field'

adjacency_data = pd.DataFrame.from_csv('adjacency.csv')
adjacency_data.index = adjacency_data.index.str.lower()
adjacency_data.columns = adjacency_data.columns.str.lower()

judge_data = pd.DataFrame.from_csv('judges.csv')
project_data = pd.DataFrame.from_csv('projects.csv')
def make_df(data):
    df = pd.DataFrame()
    df['name'] = (data['LAST NAME'] + ', ' + data['FIRST NAME']).str.lower()
    df['univ'] = data['INSTITUTION'].str.lower().astype('category')
    df['cat'] = data['CATEGORY'].str.lower().astype('category')
    df['field'] = data['SUBFIELD'].str.lower().astype('category')        
    return df.sort_values(srt)

judges = make_df(judge_data)
judges.index.names = ['judge id']
judges['mobile'] = judge_data['MOBILE COMFORT LEVEL']
projs = make_df(project_data)
projs.index.names = ['proj id']
projs = projs[projs['field'] != 'doctoral']

def make_categorical(col):
    levels = pd.Categorical(pd.concat([judges[col],projs[col]]))
    judge = pd.Categorical(judges[col], categories=levels.categories)
    proj = pd.Categorical(projs[col], categories=levels.categories)
    return levels, judge, proj

univ, judges_univ, projs_univ = make_categorical('univ')
judges['univ_code'] = judges_univ.codes
projs['univ_code'] = projs_univ.codes

cat, judges_cat, projs_cat = make_categorical('cat')
judges['cat_code'] = judges_cat.codes
projs['cat_code'] = projs_cat.codes

field, judges_field, projs_field = make_categorical('field')
judges['field_code'] = judges_field.codes
projs['field_code'] = projs_field.codes

field1 = pd.Categorical(adjacency_data.index)
field2 = pd.Categorical(adjacency_data.columns)
if(all(field.categories == field1.categories) & all(field.categories == field2.categories)):
    print("subfield lists match")
else:
    raise Exception("subfield lists do not match")

    
### Compute field distance matrix
def show():
    print()
    print("k=%d"%k)
    print("P")
    display(P,'all')
    print("dist")
    display(dist,'all')

adj = adjacency_data.values
s = adj.sum(axis=1, keepdims=True)
P = adj/s

dist = adj.copy().astype(int)
k = 1
# print("adj")
# print(aAdj)
# show()
for k in range(2,50):
    # Multiply P by itself
    P = P.dot(P)
    # Write k in all entries of Dist that are 0 and the corresponding entry of A just became non-zero
    dist[((dist==0) & (P>0))] = k
    if(dist.min() > 0): #break once Dist is full
        break
#     show()
# show()
dist[dist==0] = 999
dist[np.diag_indices_from(dist)] = 0

field_incompat = np.array([[dist[jf,pf] for pf in projs['field_code']] for jf in judges['field_code']]).astype(float)
univ_match = np.array([[ju == pu for pu in projs['univ_code']] for ju in judges['univ_code']])
field_incompat[univ_match] = np.nan


field_incompat = pd.DataFrame(field_incompat, index=judges.index, columns=projs.index)
dist = pd.DataFrame(dist, index=field.categories, columns=field.categories)

display(dist,'all')
display(field_incompat)

subfield lists match


,agriculture,"animal science, poultry science",biology,biomedical sciences,chemistry,"communication, media","computer science, engineering, systems",counseling,"criminal justice, criminology, forensics",education,...,marketing,math & statistics,nursing,organizational behavior,physics,political science,psychology,social work,sociology,wildlife & fisheries management science
agriculture,0,1,2,3,2,2,3,3,3,1,...,3,3,3,3,2,3,2,2,3,2
"animal science, poultry science",1,0,1,2,2,3,3,3,3,2,...,3,3,3,4,2,3,3,3,3,1
biology,1,1,0,1,2,3,3,3,3,2,...,3,3,2,3,2,3,3,3,3,1
biomedical sciences,2,2,1,0,1,2,3,2,2,2,...,3,3,1,3,3,2,2,2,2,2
chemistry,2,2,1,1,0,3,3,3,3,3,...,3,3,2,3,2,3,3,3,3,2
"communication, media",4,4,4,3,4,0,3,3,3,2,...,1,4,4,2,4,2,2,3,3,4
"computer science, engineering, systems",3,3,3,2,2,2,0,3,3,2,...,2,1,3,2,2,2,2,3,3,3
counseling,5,5,4,4,4,3,4,0,1,2,...,3,4,4,4,4,2,1,1,2,5
"criminal justice, criminology, forensics",5,5,4,4,4,3,4,1,0,2,...,3,4,4,4,4,1,1,1,1,5
education,4,4,4,4,4,1,4,2,2,0,...,2,4,4,3,4,2,1,1,2,4


proj id,R_3fVSZXU7HNOiIsa,R_3MMV16tlEg5mPfo,R_2fjJcJtgw37COZU,R_3p90ZtSkCO7rfWz,R_1LNLUNmSgGtujIX,R_3erhyu7B5VGpoy6,R_26m6UiXHcCsj8WD,R_1FKugNv4X226c4D,R_0qPbAUyJUchgI4p,R_10xfP6qQsG4YJ0B,...,R_3gTrQ3To6PBJrEL,R_1EWmifVasEMyujM,R_UgUJMv1EVu8aCNH,R_2pY6s13N4pylG4l,R_2wiVy8hHTFtaDji,R_1pSUJmTCKVYFVAp,R_vou2BCXk3IgQz4J,R_2ZJRCK5hj0zhs1o,R_2TufKLBP4Ojobzm,R_3FV89BXpkghpNyE
judge id,,,,,,,,,,,,,,,,,,,,,
R_3h4Pu7PsHMWGw3q,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,3.000,3.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
R_1kHYPQ4eYM6GL57,0.000,nan,1.000,1.000,1.000,nan,nan,nan,nan,nan,...,3.000,3.000,nan,nan,nan,nan,nan,nan,nan,nan
R_3D8eeW4g7GIDcZI,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,3.000,3.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
R_3JLzIuVlOVexBGt,nan,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,3.000,3.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
R_1kOcQfsuIMo9k2u,1.000,nan,0.000,0.000,0.000,nan,nan,nan,nan,nan,...,3.000,3.000,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_1DwC72wHvEepo2f,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,nan,2.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000
R_b9oVefeOBElXgGJ,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,2.000,2.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000
R_3QQzROtDVjtNzJ8,5.000,nan,5.000,5.000,5.000,nan,nan,nan,nan,nan,...,3.000,3.000,nan,nan,nan,nan,nan,nan,nan,nan


In [2]:
num_ses = 4
workload = 5
coverage = 2
num_proj_to_ses = 3
num_judge_to_proj = 2

num_judges = len(judges)

fi_best = 99999

proj_grps = projs.groupby(srt)
for ps in range(num_proj_to_ses):
    print("proj to ses attempt %d"%ps)
    proj_to_ses = proj_grps.transform(lambda x: np.random.permutation(x)).join(projs[srt])
    proj_to_ses = projs.copy()
    proj_to_ses['ses'] = np.arange(len(proj_to_ses)) % num_ses

    fi_ses_best = [99999] * num_ses
    judge_to_proj_best_ses = [[]] * num_ses
    for ses in range(num_ses):
        print("    optimizing session %d"%ses)
        judges_ses = judges.copy()
        projs_idx = proj_to_ses['ses'] == ses
        projs_ses = proj_to_ses[projs_idx]

        fi_source = field_incompat.loc[judges_ses.index.values, projs_ses.index.values]
        for jp in range(num_judge_to_proj):
            print("        judge to proj attempt %d"%jp)
            judge_to_proj = fi_source.copy()
            judge_to_proj[:] = np.nan
            fi_rec = fi_source.values.copy()
            
            sh = judge_to_proj.shape
            judge_work = np.ones(sh[0])*workload
            proj_cover = np.ones(sh[1])*coverage            
            
            while True:
                urgency = np.nanmean(fi_rec,axis=0)
                urg_max = np.nanmax(urgency)                
                if np.isnan(urg_max):
#                     print("no columns left")
                    break
                col_idxs = np.where(urgency == urg_max)[0]                
                col_idx = np.random.choice(col_idxs)
                col = fi_rec[:,col_idx]
                col_min = np.nanmin(col)
                if np.isnan(col_min):
                    print("no rows left")
                    break
                row_idxs = np.where(col == col_min)[0]
                row_idx = np.random.choice(row_idxs)
                judge_to_proj.iloc[row_idx,col_idx] = fi_rec[row_idx,col_idx]
                fi_rec[row_idx,col_idx] = np.nan
                judge_work[row_idx] -= 1
                if(judge_work[row_idx] <= 0):
                    fi_rec[row_idx,:] = np.nan
                proj_cover[col_idx] -= 1
                if(proj_cover[col_idx] <= 0):
                    fi_rec[:,col_idx] = np.nan

            fi_ses = np.nansum(judge_to_proj)
            print("        completed judge to proj %d"%jp)
            print("        new fi_ses = %f"%fi_ses)
            if(fi_ses < fi_ses_best[ses]):
                print("        better than old fi_ses_best = %f"%fi_ses_best[ses])
                fi_ses_best[ses] = fi_ses
                judge_to_proj_best_ses[ses] = pd.DataFrame(judge_to_proj, index=fi_source.index, columns = fi_source.columns)
            else:
                print("        not better than old fi_ses_best = %f"%fi_ses_best[ses])
            print()
    fi = sum(fi_ses_best)
    print("    completed proj to ses %d"%ps)
    print("    new fi = %f"%fi)    
    if(fi < fi_best):
        print("    better than old fi_best = %f"%fi_best)
        fi_best = fi
        proj_to_ses_best = proj_to_ses.copy()
        judge_to_proj_best = judge_to_proj_best_ses.copy()
    else:
        print("    not better than old fi_best = %f"%fi_best)

proj to ses attempt 0
    optimizing session 0
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 30.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1


C:\Users\scook\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice
C:\Users\scook\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: All-NaN slice encountered


        completed judge to proj 1
        new fi_ses = 30.000000
        not better than old fi_ses_best = 30.000000

    optimizing session 1
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 31.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to proj 1
        new fi_ses = 31.000000
        not better than old fi_ses_best = 31.000000

    optimizing session 2
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 28.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to proj 1
        new fi_ses = 28.000000
        not better than old fi_ses_best = 28.000000

    optimizing session 3
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 27.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to p

In [48]:
#This is not nice code.  Hope you can make it better.

projs = proj_to_ses_best
judge_to_proj = judge_to_proj_best
def get_assign(col):
    x = pd.Series(col.dropna().index)
    return x
judges_assign = [jp.apply(get_assign,axis=1) for jp in judge_to_proj]
projs_assign = [jp.T.apply(get_assign,axis=1).rename(columns = lambda x : 'judge_' + str(x)) for jp in judge_to_proj]

projs_assign[1]
projs = projs.merge(pd.concat(projs_assign),left_index=True,right_index=True,how='left')
projs
idx = pd.IndexSlice
cross = projs.iloc[:,[3,8,9]]
cross = cross.merge(judges[['field']],right_index=True, left_on='judge_0')
cross = cross.merge(judges[['field']],right_index=True, left_on='judge_1')
cross
#     ['judge_0_field'] = judges.loc[cross['judge_0'],'field']
# projs = projs.merge(projs_assign[0],left_index=True,right_index=True,how='left')
# projs.merge(judges[['field']],right_index=True, left_on='judge_0')[['field_x','field_y']]#, how='left')

,field_x,judge_0,judge_1,field_y,field
proj id,,,,,
R_3fVSZXU7HNOiIsa,agriculture,R_3h4Pu7PsHMWGw3q,R_3D8eeW4g7GIDcZI,agriculture,agriculture
R_2YJQ5wjXFH7CR00,"geosciences, geology, hydrology, geophysics",R_3h4Pu7PsHMWGw3q,R_1FETrLE2PmkRdk2,agriculture,"geosciences, geology, hydrology, geophysics"
R_2azPtAMlz9152MU,"geosciences, geology, hydrology, geophysics",R_2qCVLHWhdCvDkPY,R_1FETrLE2PmkRdk2,biology,"geosciences, geology, hydrology, geophysics"
R_3ME6Io0fRu5Lk3L,"geosciences, geology, hydrology, geophysics",R_3oBHY6bmh6ZRCQx,R_1FETrLE2PmkRdk2,biology,"geosciences, geology, hydrology, geophysics"
R_SOuSufsrs5sc6Nr,chemistry,R_3fV7Jg4bBLqspgB,R_1FETrLE2PmkRdk2,chemistry,"geosciences, geology, hydrology, geophysics"
R_2dmTAOhsT492jhO,"geosciences, geology, hydrology, geophysics",R_3fV7Jg4bBLqspgB,R_1FETrLE2PmkRdk2,chemistry,"geosciences, geology, hydrology, geophysics"
R_2S7fpDi2ArVVO4Y,"geosciences, geology, hydrology, geophysics",R_1Pcwjkecr8aAIgo,R_1FETrLE2PmkRdk2,"engineering - environmental, biological, agric...","geosciences, geology, hydrology, geophysics"
R_1puMTvJRHoTkej6,"geosciences, geology, hydrology, geophysics",R_OIKjhEVOmcbjWq5,R_1FETrLE2PmkRdk2,biology,"geosciences, geology, hydrology, geophysics"
R_3MMV16tlEg5mPfo,agriculture,R_3D8eeW4g7GIDcZI,R_3JLzIuVlOVexBGt,agriculture,agriculture


In [7]:
for (ses, jp) in enumerate(judge_to_proj):
    judges_assign = jp.apply(get_assign,axis=1)
    projs_assign = jp.T.apply(get_assign,axis=1)
    
    
#     print(judges_assign)
    
#     judges_assign = pd.DataFrame(q.tolist())#, index=q.index)
#     display(judges_assign)
#     judges = judges.merge(judges_assign, left_index=True, right_index=True)
    
#     q = jp.copy().T
    
    print(projs_assign)#.apply(pd.Series))
#     projs_assign = pd.DataFrame(q.tolist())#, index=q.index)
# #     display(proj_assign)
#     print(projs_assign.index)
#     print(projs.index)
#     projs = projs.merge(projs_assign,left_index=True,right_index=True)
# #     projs = pd.merge(projs, projs_assign, left_index=True, right_index=True)
    
    
    
#     judges = jp.copy()
    
#     jp = jp.notnull()
#     jp.index.values[jp]
#     print(jp.iloc[0])
#     print(jp)
#     x = jp.apply(lambda r: r, axis=0)
#     print(x)
#     print(proj)
#     bool = jp.notnull()
#     print(bool.dot(proj))
#     labels = bool.columns.names
#     q = jp.apply(get_judges,axis=0)
#     qdf = pd.DataFrame(q.tolist())#,index=jp.index)
#     display(qdf)
#     jp.index[bool]
#     display(jp.T.dropna())

display(projs)

                                   0                  1                  2  \
judge id                                                                     
R_3h4Pu7PsHMWGw3q  R_3fVSZXU7HNOiIsa  R_2YJQ5wjXFH7CR00                NaN   
R_1kHYPQ4eYM6GL57                NaN                NaN                NaN   
R_3D8eeW4g7GIDcZI  R_3fVSZXU7HNOiIsa                NaN                NaN   
R_3JLzIuVlOVexBGt  R_0qPbAUyJUchgI4p  R_D1z9C1G8mJyH1nj                NaN   
R_1kOcQfsuIMo9k2u  R_1LNLUNmSgGtujIX                NaN                NaN   
R_3D60VjC4WTKYczG                NaN                NaN                NaN   
R_1QykpZxmA0aT3d8  R_1LNLUNmSgGtujIX                NaN                NaN   
R_VU7Lb4Fiqa8zGoN  R_0qPbAUyJUchgI4p                NaN                NaN   
R_3r2IXneTMK5n0Ci  R_9MK0yGmMyvRnofn                NaN                NaN   
R_qVmiVRWiX1daU5r  R_3phDk9LzMpnNoFg  R_8B0IfFpnRxtW4ol  R_31cQynw7ijFfY5H   
R_2rQOdbpMke5JZ97                NaN                NaN         

                                   0                  1
proj id                                                
R_3MMV16tlEg5mPfo  R_3D8eeW4g7GIDcZI  R_3JLzIuVlOVexBGt
R_3erhyu7B5VGpoy6  R_VU7Lb4Fiqa8zGoN  R_1C2ryxvmhXBTISf
R_10xfP6qQsG4YJ0B  R_VU7Lb4Fiqa8zGoN  R_3oBHY6bmh6ZRCQx
R_1jV9oNnBUT88oqi  R_2cB1mU5QVJoGmLb  R_2WPeMAJGFEzzmgm
R_3D0lo7gQevr6aPB  R_qVmiVRWiX1daU5r  R_2rQOdbpMke5JZ97
R_1DO2CdUFbRU5Lm9  R_3r2IXneTMK5n0Ci  R_qVmiVRWiX1daU5r
R_1isVrfIwQiB0w0C  R_qVmiVRWiX1daU5r  R_2rQOdbpMke5JZ97
R_9mfyLzqmQ7qXvZD  R_3r2IXneTMK5n0Ci  R_2WPeMAJGFEzzmgm
R_2YDNK7KHevvhgw5  R_2cB1mU5QVJoGmLb  R_3oBHY6bmh6ZRCQx
R_3EoGkrACFBzjJfw  R_2cB1mU5QVJoGmLb  R_3oBHY6bmh6ZRCQx
R_2t6a9dfxb0EAXrc  R_syZDasKuSfQMQ4F  R_2YtBdyWtz1xKRtc
R_3HOCkHN2U4E8sTj  R_syZDasKuSfQMQ4F  R_2YtBdyWtz1xKRtc
R_2SpZ8EiRsUy9WOo  R_3fV7Jg4bBLqspgB  R_28AHV79JuU0zwUW
R_1dgmDJmVorUGtcj  R_8HrvMsXFeqqdgtj  R_28AHV79JuU0zwUW
R_1OlXFynLt03cp6J  R_2YtBdyWtz1xKRtc  R_3fV7Jg4bBLqspgB
R_29mMkji2Wnfxn1g  R_2YtBdyWtz1xKRtc  R_3fV7Jg4b

                                   0                  1                  2  \
judge id                                                                     
R_3h4Pu7PsHMWGw3q                NaN                NaN                NaN   
R_1kHYPQ4eYM6GL57                NaN                NaN                NaN   
R_3D8eeW4g7GIDcZI  R_2uIZInLZQW9weeW                NaN                NaN   
R_3JLzIuVlOVexBGt                NaN                NaN                NaN   
R_1kOcQfsuIMo9k2u  R_3p90ZtSkCO7rfWz                NaN                NaN   
R_3D60VjC4WTKYczG                NaN                NaN                NaN   
R_1QykpZxmA0aT3d8                NaN                NaN                NaN   
R_VU7Lb4Fiqa8zGoN  R_3p90ZtSkCO7rfWz  R_1FKugNv4X226c4D  R_2lvCGvGnWv15ZhT   
R_3r2IXneTMK5n0Ci  R_D1RTCEB1aQaJAJz  R_1pKwOZvPuBJolUg  R_2zZ5mBy2zJQ6AIB   
R_qVmiVRWiX1daU5r                NaN                NaN                NaN   
R_2rQOdbpMke5JZ97  R_TdvFBgV2Yw5Ie8V  R_2zZ5mBy2zJQ6AIB  R_1H6Od

,name,univ,cat,field,univ_code,cat_code,field_code,ses
proj id,,,,,,,,
R_3fVSZXU7HNOiIsa,"grigoryan, sona",texas a&m university - commerce,business & computer information systems,agriculture,5,1,0,0
R_3MMV16tlEg5mPfo,"robinson, hailey",tarleton state university,agriculture,agriculture,1,0,0,1
R_2fjJcJtgw37COZU,"jackson, asja",prairie view a&m university,agriculture,"animal science, poultry science",0,0,1,2
R_3p90ZtSkCO7rfWz,"brown, nicole",west texas a&m university,agriculture,"animal science, poultry science",11,0,1,3
R_1LNLUNmSgGtujIX,"preston, rebecca",texas a&m university - kingsville,agriculture,"animal science, poultry science",8,0,1,0
...,...,...,...,...,...,...,...,...
R_1pSUJmTCKVYFVAp,"hall, jared",tarleton state university,environmental science,wildlife & fisheries management science,1,4,38,2
R_vou2BCXk3IgQz4J,"mote, kelton",tarleton state university,environmental science,wildlife & fisheries management science,1,4,38,3
R_2ZJRCK5hj0zhs1o,"hopkins, melissa",tarleton state university,agriculture,wildlife & fisheries management science,1,0,38,0


In [3]:
def f(c):
    judge = c.dropna().index.tolist()
    return judge
q = judge_to_proj_best[0]
c = q.apply(f,axis=0)
# c = pd.DataFrame(c)
c = pd.DataFrame(c.tolist(),index=c.index)
c.index.rename('id',inplace=True)
c
projs
projs = projs.merge(c,left_index=True,right_index=True)
projs = projs.merge(proj_to_ses_best,left_index=True,right_index=True)
projs

,name_x,univ_x,cat_x,field_x,univ_code_x,cat_code_x,field_code_x,0,1,name_y,univ_y,cat_y,field_y,univ_code_y,cat_code_y,field_code_y,ses
id,,,,,,,,,,,,,,,,,
R_3fVSZXU7HNOiIsa,"grigoryan, sona",texas a&m university - commerce,business & computer information systems,agriculture,5,1,0,R_3h4Pu7PsHMWGw3q,R_1kHYPQ4eYM6GL57,"grigoryan, sona",texas a&m university - commerce,business & computer information systems,agriculture,5,1,0,0
R_1LNLUNmSgGtujIX,"preston, rebecca",texas a&m university - kingsville,agriculture,"animal science, poultry science",8,0,1,R_3D60VjC4WTKYczG,R_1QykpZxmA0aT3d8,"preston, rebecca",texas a&m university - kingsville,agriculture,"animal science, poultry science",8,0,1,0
R_0qPbAUyJUchgI4p,"lahmon, alyssa",tarleton state university,agriculture,"animal science, poultry science",1,0,1,R_VU7Lb4Fiqa8zGoN,R_qVmiVRWiX1daU5r,"lahmon, alyssa",tarleton state university,agriculture,"animal science, poultry science",1,0,1,0
R_3shU3hGqUO9PJ0x,"solis, joshua",texas a&m university - san antonio,life science,biology,9,6,2,R_3r2IXneTMK5n0Ci,R_2cB1mU5QVJoGmLb,"solis, joshua",texas a&m university - san antonio,life science,biology,9,6,2,0
R_3phDk9LzMpnNoFg,"lazo, krista",texas a&m university - san antonio,life science,biology,9,6,2,R_3r2IXneTMK5n0Ci,R_2WPeMAJGFEzzmgm,"lazo, krista",texas a&m university - san antonio,life science,biology,9,6,2,0
R_8B0IfFpnRxtW4ol,"debose-boyd, ryan",texas a&m university - commerce,life science,biology,5,6,2,R_qVmiVRWiX1daU5r,R_OIKjhEVOmcbjWq5,"debose-boyd, ryan",texas a&m university - commerce,life science,biology,5,6,2,0
R_9MK0yGmMyvRnofn,"migl, chesney",texas a&m university - kingsville,life science,biology,8,6,2,R_qVmiVRWiX1daU5r,R_2qCVLHWhdCvDkPY,"migl, chesney",texas a&m university - kingsville,life science,biology,8,6,2,0
R_210PQqQpojF9AHi,"warford, khira",texas a&m university - san antonio,life science,biology,9,6,2,R_3r2IXneTMK5n0Ci,R_2WPeMAJGFEzzmgm,"warford, khira",texas a&m university - san antonio,life science,biology,9,6,2,0
R_3k6UYN56Ox3HOHY,"szteiter, shelby",texas a&m university - kingsville,physical science,biology,8,8,2,R_2rQOdbpMke5JZ97,R_2qCVLHWhdCvDkPY,"szteiter, shelby",texas a&m university - kingsville,physical science,biology,8,8,2,0


In [4]:
def f(r):
    proj = r.dropna().index.tolist()
    return proj

def g(ses):
    q = judge_to_proj_best[ses]
    q = q.apply(f,axis=1)
    q = pd.DataFrame(q.tolist(),index = q.index)
    q.index.rename('judge id',inplace=True)
    return q
temp = [g(ses).T for ses in range(num_ses)]
judge_assign = pd.concat(temp).T
# temp[0]
judge_assign.shape

(94, 18)

In [20]:
judge_to_proj_best[0].iloc[:,3].dropna().index.tolist()

['R_2qCVLHWhdCvDkPY', 'R_2WPeMAJGFEzzmgm']

In [6]:
display(proj_to_ses_best.sort_values(['field','ses']),'all')
# display(judge_to_proj_best,'all')

,name,univ,cat,field,univ_code,cat_code,field_code,ses
id,,,,,,,,
R_3fVSZXU7HNOiIsa,"grigoryan, sona",texas a&m university - commerce,business & computer information systems,agriculture,5,1,0,0
R_3MMV16tlEg5mPfo,"robinson, hailey",tarleton state university,agriculture,agriculture,1,0,0,1
R_1LNLUNmSgGtujIX,"preston, rebecca",texas a&m university - kingsville,agriculture,"animal science, poultry science",8,0,1,0
R_0qPbAUyJUchgI4p,"lahmon, alyssa",tarleton state university,agriculture,"animal science, poultry science",1,0,1,0
R_3erhyu7B5VGpoy6,"stotz, miranda",tarleton state university,agriculture,"animal science, poultry science",1,0,1,1
R_10xfP6qQsG4YJ0B,"payne, kristen",tarleton state university,agriculture,"animal science, poultry science",1,0,1,1
R_2fjJcJtgw37COZU,"jackson, asja",prairie view a&m university,agriculture,"animal science, poultry science",0,0,1,2
R_26m6UiXHcCsj8WD,"wommack, mikayla",tarleton state university,agriculture,"animal science, poultry science",1,0,1,2
R_2QMNyq5txUsv9K8,"herrygers, kallie",tarleton state university,agriculture,"animal science, poultry science",1,0,1,2


In [ ]:
5+5

In [29]:
row_idx = 5
col_idx = 2
proj_id = judge_to_proj.columns[col_idx]
judge_id = judge_to_proj.index[row_idx]
proj_id
judge_id
judge_to_proj.loc[judge_id,proj_id]=33
display(np.vstack(np.unique(judge_to_proj,return_counts=True)).T,'all')
# judge_to_proj

,0,1
0,0.000,855.000
1,1.000,3248.000
2,2.000,3818.000
3,3.000,2548.000
4,4.000,1938.000
5,10.000,592.000
6,17.000,10.000
7,33.000,10.000
8,nan,1.000
9,nan,1.000


In [30]:
num_ses = 4
workload = 5
coverage = 2
num_proj_to_ses = 3
num_judge_to_proj = 2

num_judges = len(judges)

fi_best = 99999

# proj_grps = projs.groupby(srt)
for ps in range(num_proj_to_ses):
    print("proj to ses attempt %d"%ps)
#     proj_to_ses = proj_grps.transform(lambda x: np.random.permutation(x)).join(projs[srt])
    proj_to_ses = projs.copy()
    proj_to_ses['ses'] = np.arange(len(proj_to_ses)) % num_ses

    fi_ses_best = [99999] * num_ses
    judge_to_proj_best_ses = [[]] * num_ses
    for ses in range(num_ses):
        print("    optimizing session %d"%ses)
        judges_ses = judges.copy()
        projs_idx = proj_to_ses['ses'] == ses
        projs_ses = proj_to_ses[projs_idx]        

        fi_source = field_incompat.loc[judges_ses.index.values, projs_ses.index.values]
        fi_source = pd.concat([fi_source]*workload)
        fi_source = pd.concat([fi_source.T]*coverage).T

        for jp in range(num_judge_to_proj):
            print("        judge to proj attempt %d"%jp)
            judge_to_proj = fi_source.astype(float)   
            urgency = judge_to_proj.sum(axis=0)
#             display(urgency,'all')

            while True:
                urg_max = np.nanmax(urgency.values)
#                 print(urg_max)
                if np.isnan(urg_max):
#                     print("no columns left")
                    break
                col_idxs = np.where(urgency == urg_max)[0]                
                col_idx = np.random.choice(col_idxs)
                
                col = judge_to_proj.iloc[:,col_idx]
                col_min = np.nanmin(col.values)
                if np.isnan(col_min):
                    print("no rows left")
                    break
                row_idxs = np.where(col == col_min)[0]
                row_idx = np.random.choice(row_idxs)
                proj_id = judge_to_proj.columns[col_idx]
                judge_id = judge_to_proj.index[row_idx]

                temp = judge_to_proj.iloc[row_idx,col_idx]
                urgency -= judge_to_proj.iloc[row_idx,:]
                judge_to_proj.iloc[row_idx,:] = np.nan
                judge_to_proj.iloc[:,col_idx] = np.nan
                judge_to_proj.loc[judge_id,proj_id] = np.nan
                urgency.iloc[col_idx] = np.nan
                judge_to_proj.iloc[row_idx,col_idx] = temp

            fi_ses = np.nansum(judge_to_proj)
            print("        completed judge to proj %d"%jp)
            print("        new fi_ses = %f"%fi_ses)
            if(fi_ses < fi_ses_best[ses]):
                print("        better than old fi_ses_best = %f"%fi_ses_best[ses])
                fi_ses_best[ses] = fi_ses
                judge_to_proj_best_ses[ses] = pd.DataFrame(judge_to_proj, index=fi_source.index, columns = fi_source.columns)
            else:
                print("        not better than old fi_ses_best = %f"%fi_ses_best[ses])
            print()
    fi = sum(fi_ses_best)
    print("    completed proj to ses %d"%ps)
    print("    new fi = %f"%fi)    
    if(fi < fi_best):
        print("    better than old fi_best = %f"%fi_best)
        fi_best = fi
        proj_to_ses_best = proj_to_ses.copy()
        judge_to_proj_best = judge_to_proj_best_ses.copy()
    else:
        print("    not better than old fi_best = %f"%fi_best)

proj to ses attempt 0
    optimizing session 0
        judge to proj attempt 0


C:\Users\scook\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: All-NaN slice encountered


        completed judge to proj 0
        new fi_ses = 5307.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to proj 1
        new fi_ses = 6515.000000
        not better than old fi_ses_best = 5307.000000

    optimizing session 1
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 5417.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to proj 1
        new fi_ses = 6339.000000
        not better than old fi_ses_best = 5417.000000

    optimizing session 2
        judge to proj attempt 0
        completed judge to proj 0
        new fi_ses = 7438.000000
        better than old fi_ses_best = 99999.000000

        judge to proj attempt 1
        completed judge to proj 1
        new fi_ses = 8597.000000
        not better than old fi_ses_best = 7438.000000

    optimizing session 3
        judge to proj attempt 0
        co

In [9]:
def make_all_post_to_ses():
    """
    Generate all possible ways to assign posters to sessions.  Two steps:
    1. Select num_ses posters in increasing order to be the first posters assigned to each ses.
    2. Fill the remaining slots in each ses
    
    Explanation - Step 1 prevents redundancy seen in earlier, more naive versions of this algorithm.
    Example - Suppose num_ses = 2 and num_post = 4.  We do not want to see both
    post_ses = [[0,1],[2,3]] and post_ses = [[2,3],[0,1]].  These are redundant.  
    If you flip ses 0 and ses 1, it is the same (except posters 2&3 present in the earlier session).    
    """

    N = num_post # tracks number of posters not yet assigned to a session
    k = num_ses
    first_post_idx_gen = it.combinations(range(N), k) #generates all ways to select first for each ses in increading order
    N -= k # there are now num_ses fewer poster left to assign
    other_post_idx = []
    for ses in range(num_ses):
        k = num_post_ses[ses]-1 # Number of slots still open in this ses.  Recall - there's already 1 assigned.
        other_post_idx.append(it.combinations(range(N), k)) #generates all ways to fill open slots
        N -= k # fewer posters left to assign
    other_post_idx_gen = it.product(*other_post_idx) #combines the separate generators for each session into one big generator
    return first_post_idx_gen, other_post_idx_gen

#########################################################
# The code below is just a consistency check of the code above.  It computes the total number
# of ways to assign posters to sessions in two distinct ways.  Should produce same answer.
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
s = 0
for _ in first_post_idx_gen:
    s += 1
t = 0
for _ in other_post_idx_gen:
    t += 1
print(s*t)

# count a different way to verify
cs = np.array(num_post_ses) - 1
cs = cs.cumsum().tolist()
cs.insert(0,0)
N = num_post
u = binom(N,num_ses)
N -= num_ses
v = [binom(N-cs[i],num_post_ses[i]-1) for i in range(num_ses)]
print(int(u*np.prod(v)))

#########################################################
#Assign judges to posters
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
best_post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
best_judg_post = [[]]*num_ses
best_fi_score_tot = 999999

for first_post_idx in first_post_idx_gen:
    
    for other_post_idx in other_post_idx_gen:
        
        L = list(range(num_post))
        post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
        
        # We will walk first_post_idx and other_post_idx backward because we will use the pop method to remove
        # used posters from L.  When an element is popped from L, all elements to its right slide
        # one slot to the left.  But all elements to its left do not move.  Thus, we want to pop from
        # right to left so indexing is consistent.  Example, let L=[0,1,2,3].  let post_idx = [0,2]
        # Thus, we expect to pop 0 and 2 from L.  Suppose we pop 0 first.  Now L = [1,2,3].  When we
        # pop 2, we actually get 3, because 3 is now in slot 2.  However, if we pop 2 first,
        # L = [0,1,2].  Now we pop 0 and get 0.  This is the desired behavior.
        # Recall [::-1] reverses a list        
        for (ses, post_idx) in enumerate(first_post_idx[::-1]):
            # we use -(ses+1) so that the smallest index goes into the first ses and the largest goes into the last ses.
            # We also put into the last slot of each ses (not first slot as discussed above).
            # This is merely a cosmetic preference - makes posters within the same ses appear
            # in a consistent order. But the order of posters within the same ses does not actually matter.
            post_ses[-(ses+1)][-1] = L[post_idx]
            L.pop(post_idx)

        for (ses, post_idx) in enumerate(other_post_idx):
            for (i, j) in enumerate(post_idx[::-1]): # Again, reverse so pop works right (see above)
                post_ses[ses][i] = L.pop(j)
            post_ses[ses] = post_ses[ses][::-1] # Again, cosmetic.  The posters in each session
# are listed in descending order.  This reverses into ascending order.  Turn off to increase speed.
#         print(post_ses)    
    
        #########################################################
        # Assign judges to posters
        best_fi_ses = [[]] * num_ses #best fi matrix
        best_fi_score_ses = [999] * num_ses #best fi_score
        print()
        print("Optimizing poster->session %s"%post_ses)

        for (ses, post) in enumerate(post_ses):  
            print("Optimizing session %d with posters %s"%(ses,post))
            workload = np.ceil(len(post)*cover/num_judg)
#             print(workload)
            for trial in range(num_trials):
                skip_trial = False
                field_incompat_sub = field_incompat[:,post] #Extract cols for posters assigned to this ses
#                 print(field_incompat_sub)
                fi = np.zeros_like(field_incompat_sub)
                judg_work = [0]*num_judg #Tracks each judge's workload
                lot_order = np.random.permutation(len(post)) #randomize poster lottery order
                for i in range(cover): # number of "rounds" of the lottery = cover
                    if(skip_trial == True):
                        break
#                     print("round = %d"%i)
                    for col in lot_order:
#                         print("col = %d"%col)
                        c = field_incompat_sub[:,col] #Grab column for this poster
                        m = c.min()
                        if(m == 999): #If no legal judge remains, abandon this trial
                            skip_trial = True
                            break
                        rows = np.where(c == m)[0] #find all rows in this col where the smallest fi appears
#                         print(rows)
                        if(len(rows) == 1):
                            row = rows[0]
                        else:
                            row = np.random.choice(rows) #choose one such row at random
#                         print(row)
                        fi[row,col] = field_incompat_sub[row,col] #record fi for this pairing
                        judg_work[row] += 1 #increment that judge's workload tracker
                        if(judg_work[row] >= workload):
                            field_incompat_sub[row,:] = 999 #if judge has reached workload limit, fill her row with 999
                        else:
                            field_incompat_sub[row,col] = 999 #else, put 999 in the (row,col) entry only
#                         print(fi)
#                         print(field_incompat_sub)
                if(skip_trial == True):
                    continue
                fi_score = fi.sum() #total fi score for this judge->poster pairing for this ses
#                 print(field_incompat_sub)
#                 print(fi>0)
#                 print(fi)
#                 print(fi_score)
                if(fi_score < best_fi_score_ses[ses]): # if current beats prior best, keep it
                    print("current ses %d fi = %d is better than prior best ses %d fi = %d.  I'll keep it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
                    best_fi_ses[ses] = fi.copy()
                    best_fi_score_ses[ses] = fi_score
                else:
                    print("current ses %d fi = %d is not better than prior best ses %d fi = %d.  I'll ignore it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
#                 print(best_fi_score_ses[ses])
                if(best_fi_score_ses[ses] <= 1):
                    print("Found best possible j->p for ses %d.  Moving on."%ses)        
        print("Finished optimizing session %d"%ses)
        #########################################################
        # We now have the best way to assign judges0>posters under the current poster->session assignment
        # We check if it beats the best from prior poster->session assignment
        print("Finished optimizing poster->session %s"%post_ses)
        fi_score_tot = sum(best_fi_score_ses)
        if(fi_score_tot < best_fi_score_tot):
            print("current total fi = %d is better than prior best total fi = %d.  I'll keep it."%(fi_score_tot,best_fi_score_tot))
            best_post_ses = post_ses.copy()
            best_judg_post = best_fi_ses.copy()
            best_fi_score_tot = fi_score_tot            
            for (ses,(ps,jp)) in enumerate(zip(best_post_ses,best_judg_post)):
                df = pd.DataFrame(jp>0, columns=ps)
                print("session %d"%ses)
                display(df)
                print()
        else:
            print("current total fi = %d is not better than prior best total fi = %d.  I'll ignore it."%(fi_score_tot,best_fi_score_tot))
                


True

In [54]:
field_incompat

array([[  0,   0,   1, ...,   2,   2,   2],
       [  0, 999,   1, ..., 999, 999, 999],
       [  0,   0,   1, ...,   2,   2,   2],
       ..., 
       [  5, 999,   5, ..., 999, 999, 999],
       [  1, 999,   1, ..., 999, 999, 999],
       [999,   1,   1, ...,   0,   0,   0]])

In [33]:
field_incompat.index.shape

(94,)

In [37]:
field_incompat_ses.shape

(376, 122)

In [30]:
i = 0
while True:
    print(i)
    i+=1
    if i>=5:
        break

0
1
2
3
4


In [93]:
B = np.full(5,np.nan)
B
np.nanmin(B)

C:\Users\scook\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: All-NaN axis encountered
  This is separate from the ipykernel package so we can avoid doing imports until


nan

In [ ]:
B = np.full

In [84]:
A = np.arange(3).astype(float)
# A[0] = np.nan
# A[1] = np.nan
# A[2] = np.nan
print(A)
a = np.nanmin(A)
while not np.isnan(a):
    a = np.nanmin(A)
    print(a)
    A[i] = np.nan
    i += 1

[ 0.  1.  2.]
0.0


IndexError: index 5 is out of bounds for axis 0 with size 3

In [58]:
type(urgency)

numpy.ndarray

In [107]:
[9999]*5

[9999, 9999, 9999, 9999, 9999]

In [79]:
#q = projs.groupby(srt).transform(lambda x: x.sample(frac=1)).join(projs[srt])
# print(q.shape)

num_ses = 4
workload = 4
coverage = 2

num_judges = len(judges)

q = projs.copy().groupby(srt).transform(lambda x: np.random.permutation(x)).join(projs[srt])
print(q.shape)
q['ses'] = np.arange(len(q)) % num_ses
print(q.shape)
# display(q,'all')
# display(q['ses'])

# ses_grp = q.groupby('ses')
# display(ses_grp.apply(lambda x:x).shape)



ses = 1
for ses in range(num_ses):
    judges_ses = judges.copy()
    projs_ses = q[q['ses'] == ses]
    projs_ses = projs_ses
    
    fi_source = field_incompat.loc[judges_ses.index.values, projs_ses.index.values]
    fi_source = pd.concat([fi_source]*workload)
    fi_source = pd.concat([fi_source.T]*coverage).T
    
    fi_rec = fi_source.copy()
    fi_rec[:] = np.nan
    fi_source = fi_source.values.astype(float)
    
    
    urgency = fi_source.sum(axis=0)
    display(fi_source)
    
    
    while True:
#         display(fi_source)
#         print(urgency)
        try:
            col = np.nanargmax(urgency)
        except:
#             print("out of columns")
            break
        row = np.nanargmin(fi_source[:,col])
#         print(row,col)
        fi_rec.iloc[row,col] = fi_source[row,col]
        urgency -= fi_source[row,:]
        fi_source[row,:] = np.nan
        fi_source[:,col] = np.nan
        urgency[col] = np.nan
            
    print(np.nansum(fi_rec))#.values.nansum())
    display(fi_rec)
#     display(fi_rec.iloc[:,0],'all')
#     print(urgency.shape)
#     pair = np.zeros([num_judges*workload, num_projs*coverage]).astype('int')
#     display(projs)
#     display(fi)
#     print(judges_ses.index)
    
    


# for (ses,projs) in ses_grp:
#     print(ses)
#     display(projs)
#     print()

(247, 7)
(247, 8)


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,0.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,3.000,2.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000
1,0.000,1.000,10.000,2.000,2.000,2.000,2.000,2.000,2.000,10.000,...,3.000,2.000,2.000,2.000,2.000,2.000,10.000,10.000,10.000,10.000
2,0.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,3.000,2.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000
3,10.000,1.000,1.000,2.000,2.000,10.000,2.000,2.000,2.000,2.000,...,3.000,2.000,2.000,2.000,2.000,10.000,2.000,3.000,2.000,2.000
4,1.000,0.000,10.000,1.000,1.000,1.000,1.000,1.000,1.000,10.000,...,3.000,3.000,3.000,3.000,3.000,3.000,10.000,10.000,10.000,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,10.000,0.000,10.000,0.000,0.000,0.000,1.000,2.000,4.000,4.000
372,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,2.000,0.000,0.000,0.000,0.000,0.000,1.000,2.000,4.000,4.000
373,5.000,5.000,10.000,4.000,4.000,4.000,4.000,4.000,4.000,10.000,...,3.000,1.000,1.000,1.000,1.000,1.000,10.000,10.000,10.000,10.000
374,1.000,1.000,10.000,1.000,1.000,1.000,1.000,1.000,1.000,10.000,...,3.000,3.000,3.000,3.000,3.000,3.000,10.000,10.000,10.000,10.000


18.0


,R_3fVSZXU7HNOiIsa,R_1LNLUNmSgGtujIX,R_0qPbAUyJUchgI4p,R_3shU3hGqUO9PJ0x,R_3phDk9LzMpnNoFg,R_8B0IfFpnRxtW4ol,R_9MK0yGmMyvRnofn,R_210PQqQpojF9AHi,R_3k6UYN56Ox3HOHY,R_31cQynw7ijFfY5H,...,R_3DwiOaw9fGXmmUF,R_2WOte8FmD285AIq,R_232zaDrzWs5MIBr,R_3KJFmuxyiKZCcYP,R_3FWjoFULiEer7d9,R_1FScxFq5qfX5pYX,R_3VITx7SxGQ5Cpq1,R_pt6cV5YyFpRdog9,R_2pY6s13N4pylG4l,R_2ZJRCK5hj0zhs1o
R_3h4Pu7PsHMWGw3q,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kHYPQ4eYM6GL57,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3D8eeW4g7GIDcZI,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3JLzIuVlOVexBGt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kOcQfsuIMo9k2u,nan,0.000,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_1DwC72wHvEepo2f,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_b9oVefeOBElXgGJ,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3QQzROtDVjtNzJ8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1rANHDWIJ6SmhXO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,0.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,2.000,2.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000
1,10.000,10.000,10.000,2.000,2.000,2.000,2.000,2.000,10.000,10.000,...,2.000,2.000,2.000,2.000,2.000,2.000,10.000,3.000,10.000,10.000
2,0.000,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,2.000,2.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000
3,0.000,1.000,1.000,2.000,10.000,2.000,2.000,2.000,2.000,2.000,...,2.000,2.000,2.000,2.000,2.000,2.000,2.000,3.000,2.000,2.000
4,10.000,10.000,10.000,1.000,1.000,1.000,1.000,1.000,10.000,10.000,...,3.000,3.000,3.000,3.000,3.000,3.000,10.000,3.000,10.000,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,0.000,0.000,10.000,10.000,10.000,0.000,1.000,10.000,4.000,4.000
372,4.000,4.000,4.000,4.000,4.000,4.000,4.000,10.000,4.000,4.000,...,0.000,10.000,0.000,0.000,0.000,0.000,1.000,2.000,4.000,4.000
373,10.000,10.000,10.000,4.000,4.000,4.000,4.000,4.000,10.000,10.000,...,1.000,1.000,1.000,1.000,1.000,1.000,10.000,3.000,10.000,10.000
374,10.000,10.000,10.000,1.000,1.000,1.000,1.000,1.000,10.000,10.000,...,3.000,3.000,3.000,3.000,3.000,3.000,10.000,3.000,10.000,10.000


...

16.0


,R_2fjJcJtgw37COZU,R_26m6UiXHcCsj8WD,R_2QMNyq5txUsv9K8,R_1gckYcGVSVA2ux7,R_1opPT8a3KAn00R1,R_1LHBqoBotwOK2Wm,R_22R38F7Ad2j8ufo,R_3kiUooKLLaCbChh,R_NXBjKhy5XBpJK3n,R_1mjUwPisBSab6Xx,...,R_aaey8wxcdDwYpTr,R_2qws5afZMHIEYCb,R_3Ea1aI8ShDT2VDZ,R_33ydZXB15To2pmq,R_3LX8SW8mxj4OYEY,R_2CTbgvmc4zd1jE0,R_3erdUw778WTZgm0,R_1EWmifVasEMyujM,R_1pSUJmTCKVYFVAp,R_3FV89BXpkghpNyE
R_3h4Pu7PsHMWGw3q,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kHYPQ4eYM6GL57,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3D8eeW4g7GIDcZI,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3JLzIuVlOVexBGt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kOcQfsuIMo9k2u,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_1DwC72wHvEepo2f,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_b9oVefeOBElXgGJ,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3QQzROtDVjtNzJ8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1rANHDWIJ6SmhXO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
1,1.000,10.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,10.000,...,10.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,10.000,10.000
2,1.000,1.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,...,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
3,1.000,1.000,2.000,2.000,10.000,2.000,2.000,2.000,2.000,2.000,...,2.000,10.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000
4,0.000,10.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,10.000,...,10.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,10.000,10.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,4.000,0.000,10.000,10.000,0.000,0.000,1.000,1.000,4.000,4.000
372,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,...,4.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,4.000,4.000
373,5.000,10.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,10.000,...,10.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,10.000,10.000
374,1.000,10.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,10.000,...,10.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,10.000,10.000


10.0


,R_3p90ZtSkCO7rfWz,R_1FKugNv4X226c4D,R_D1RTCEB1aQaJAJz,R_TdvFBgV2Yw5Ie8V,R_1pKwOZvPuBJolUg,R_2zZ5mBy2zJQ6AIB,R_1H6OdPefE3ubEH4,R_bpSl9UQAeht0GHf,R_1PRX7GjClvimkop,R_24H5cvIZWGzWkKq,...,R_1kYLLg96i7dtJzG,R_3ML4AQnoYUFiTSp,R_3NJ1cE0LpCZK4H6,R_3Dowl6dm3meWcM0,R_OkR0ZmDHwcAmE8N,R_1pRNLxqiusgCJJa,R_2TRxmN0Osz265Za,R_1Hi4SFoYOjX8PI5,R_UgUJMv1EVu8aCNH,R_vou2BCXk3IgQz4J
R_3h4Pu7PsHMWGw3q,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kHYPQ4eYM6GL57,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3D8eeW4g7GIDcZI,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3JLzIuVlOVexBGt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_1kOcQfsuIMo9k2u,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_1DwC72wHvEepo2f,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_b9oVefeOBElXgGJ,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
R_3QQzROtDVjtNzJ8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,0.000,nan,nan
R_1rANHDWIJ6SmhXO,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [46]:
    fi_score = fi_rec.values.sum()
    display(fi_rec.iloc[:,1].describe())
    


,R_1FKugNv4X226c4D
count,376.000
mean,0.000
std,0.000
min,0.000
25%,0.000
50%,0.000
75%,0.000
max,0.000


In [76]:
field.categories

Index(['agriculture', 'animal science, poultry science', 'biology',
       'biomedical sciences', 'chemistry', 'communication, media',
       'computer science, engineering, systems', 'counseling',
       'criminal justice, criminology, forensics', 'education',
       'education - special ed', 'engineering - civil',
       'engineering - electrical',
       'engineering - environmental, biological, agricultural',
       'engineering - mechanical', 'engineering - other',
       'english, literature', 'entomology',
       'environmental studies - plant sciences, forestry',
       'environmental studies - soil sciences', 'geography, gis',
       'geosciences, geology, hydrology, geophysics', 'health - public, human',
       'history', 'human resource management, development',
       'kinesiology, sport sciences', 'language, linguistics', 'management',
       'management information systems', 'marketing', 'math & statistics',
       'nursing', 'organizational behavior', 'physics', 'politic

In [68]:
judges.index

Index(['R_3h4Pu7PsHMWGw3q', 'R_1kHYPQ4eYM6GL57', 'R_3D8eeW4g7GIDcZI',
       'R_3JLzIuVlOVexBGt', 'R_1kOcQfsuIMo9k2u', 'R_3D60VjC4WTKYczG',
       'R_1QykpZxmA0aT3d8', 'R_VU7Lb4Fiqa8zGoN', 'R_3r2IXneTMK5n0Ci',
       'R_qVmiVRWiX1daU5r', 'R_2rQOdbpMke5JZ97', 'R_2cB1mU5QVJoGmLb',
       'R_2qCVLHWhdCvDkPY', 'R_3oBHY6bmh6ZRCQx', 'R_2WPeMAJGFEzzmgm',
       'R_OIKjhEVOmcbjWq5', 'R_syZDasKuSfQMQ4F', 'R_8HrvMsXFeqqdgtj',
       'R_2YtBdyWtz1xKRtc', 'R_3fV7Jg4bBLqspgB', 'R_28AHV79JuU0zwUW',
       'R_3hg1jtVRAMSrUNi', 'R_10TS06cUjvdUIrF', 'R_3oZSvR1j3nBNAH8',
       'R_2xWXAwQWEf8psQg', 'R_2WMrQs41Rq862L9', 'R_1gSZlAKc7NeeayG',
       'R_20TzKuKa7BsZJXY', 'R_2Sf2wDr21oqMYhv', 'R_2f96xBWeHgak4I5',
       'R_294wtVRlj5Vvtkq', 'R_247ydTZ4bAgkLXO', 'R_2PbwYq9uhrRSRwT',
       'R_2ykGluRE0jQaLiv', 'R_3lyjq7rnbqKG0LE', 'R_1JWIGchZsitsoIY',
       'R_3CTad08gKc24Lb6', 'R_1pEwVDWQkquWmHZ', 'R_OdQJEo237GnCSOt',
       'R_1IGjW2meGLxVpNx', 'R_1H2B2wF9Ujo4e7Y', 'R_2RNSrbXChtFZiiS',
       'R_1Pcwjkecr8

### Assumptions/requirements
- 9 categories
- many subfields
- Each judge and project self-identifies both a category and subfield
- Judges stay in the same category througout (this is not strictly necessary, but simplifies the process a lot)
- Thus, the judge-project pairing process works for each category independently.  We will describe this process for a single category.
- The projects (in a single category) should be evenly split among the sessions. If an uneven number, assign fewer to the first session (for late arrivers, like International)
- Judges should not judge posters from their home university.


### Setup tasks
- Determine reasonable lower and upper bounds for proster per session workload for a single judge.
- Determine $Adj$ - the adjacency matrix of subfields.
  - Let $nf$ = number of subfields.  $Adj$ is the $nf \times nf$ matrix where $a_{ij} = 1$ if subfields $i$ and $j$ are adjacent and 0 else.
- Let $P$ be row stochastic version of $A$.  (Sum across each row then divide all entries by their row's sum).  We are doing a simple random walk on the subfield adjancy graph.
- Compute $Dist$ - the distance matrix between subfields.
    1. Initialize $Dist$ as a copy of $Adj$.
    2. Compute $P^2$.  If an entry switches from 0 to non-zero, write 2 in the corresponding entry of $Dist$.  These subfields are 2 jumps apart.
    3. Compute $P^3$.  If an entry switches from 0 to non-zero, write 3 in the corresponding entry of $Dist$.  These subfields are 3 jumps apart.
    4. Continue until (hopefully) all entries of $P^k$ (and thus $Dist$) are non-zero.  $Dist$ is the subfield distance matrix.
- Compute field_incompat - the field incompatibiity matrix
    - field_incompat is the num_judg $\times$ num_post matrix where field_incompat[j,p] = Dist[field_judg[j],field_post[p]].
    - Unless judge[j] and post[p] are from the same university.  In this case, write 999 in that entry.



### Iterative steps

There are 2 major steps: assign posters to sessions AND assign judges to posters

1. Randomly assign posters$\to$sessions such that:
    - All sessions feature the same number of projects if possible.  If not evenly divisible, assign assign one additional poster to each session, starting with the last.  (for late arrivers, like International).
    - We plan to iterate exhaustively over ALL ways to do so.
    - Code is techincal - see inline comments
2. Randomly assign judges$\to$posters such that:
    1. All judges have same workload (#posters to judge)
    2. All posters will receive same number of judgings
    3. Description of algorithm
        1. Use a lottery system where posters select their judges.
        1. Randomize poster selection order. 
        1. Make a copy of big field_incompat with only columns for posters assigned to the session we are looking at.
        1. First poster finds list of ALL judges with minimal field_incompat (aka all rows in its column of field_incompat containing the current min in that column.
        1. Randomly select one of them.
        1. Record that field_incompat score into a second matrix called fi.
        1. Replace that entry in field_incompat with 999.
        1. Increment that judge's workload counter.
        1. If this assignment completes that judge's workload, fill that judge's row with 999
        1. Repeat for all posters in the randomized order selected earlier.  When all all posters have selected once, round 1 of the lottery is done.
        1. Perform more rounds until posters have the desired number of judgings.
        1. Compute field incompatibility score (fis) by summing fi (where we moved field_incompat scores before replacing with 999).  Keep track of best fis found.
        1. Repeat some reasonably large number of times (called num_trials), using the SAME poster$\to$session assignment but new judge$\to$poster assignement
5. Repeat with next poster$\to$session assignment.
6. Repeat for all 9 major categories

,name,univ,cat,univ_code,cat_code,field_code,field,ses
RESPONSE ID,,,,,,,,
R_3fVSZXU7HNOiIsa,"robinson, hailey",tarleton state university,agriculture,1,0,0,agriculture,0
R_3MMV16tlEg5mPfo,"grigoryan, sona",texas a&m university - commerce,business & computer information systems,5,1,0,agriculture,1
R_2QMNyq5txUsv9K8,"brown, nicole",west texas a&m university,agriculture,11,0,1,"animal science, poultry science",2
R_10xfP6qQsG4YJ0B,"stotz, miranda",tarleton state university,agriculture,1,0,1,"animal science, poultry science",3
R_1FKugNv4X226c4D,"herrygers, kallie",tarleton state university,agriculture,1,0,1,"animal science, poultry science",0
R_26m6UiXHcCsj8WD,"wommack, mikayla",tarleton state university,agriculture,1,0,1,"animal science, poultry science",1
R_0qPbAUyJUchgI4p,"jackson, asja",prairie view a&m university,agriculture,0,0,1,"animal science, poultry science",2
R_1LNLUNmSgGtujIX,"baker, jonathan",tarleton state university,agriculture,1,0,1,"animal science, poultry science",3
R_3p90ZtSkCO7rfWz,"payne, kristen",tarleton state university,agriculture,1,0,1,"animal science, poultry science",0


Help on method groupby in module pandas.core.generic:

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, **kwargs) method of pandas.core.frame.DataFrame instance
    Group series using mapper (dict or key function, apply given function
    to group, return result as series) or by a series of columns.
    
    Parameters
    ----------
    by : mapping, function, str, or iterable
        Used to determine the groups for the groupby.
        If ``by`` is a function, it's called on each value of the object's
        index. If a dict or Series is passed, the Series or dict VALUES
        will be used to determine the groups (the Series' values are first
        aligned; see ``.align()`` method). If an ndarray is passed, the
        values are used as-is determine the groups. A str or list of strs
        may be passed to group by the columns in ``self``
    axis : int, default 0
    level : int, level name, or sequence of such, default None
    

In [7]:
# judges.sort_values('subfield',inplace=True)
proj_cat.describe()


,counts,freqs
categories,,
agriculture,27,0.109
business & computer information systems,15,0.061
education,27,0.109
engineering & computer science,34,0.138
environmental science,14,0.057
humanities or social science,55,0.223
life science,41,0.166
mathematics,11,0.045
physical science,23,0.093


In [21]:
univ

[texas a&m university - kingsville, prairie view a&m university, texas a&m university - commerce, prairie view a&m university, texas a&m university - college station, ..., prairie view a&m university, prairie view a&m university, texas a&m university - kingsville, tarleton state university, west texas a&m university]
Length: 341
Categories (12, object): [prairie view a&m university, tarleton state university, texas a&m international university, texas a&m university - central texas, ..., texas a&m university - kingsville, texas a&m university - san antonio, texas a&m university - texarkana, west texas a&m university]

In [1]:
set1 = set(adjacency_data.index)
set2 = set(adjacency_data.columns)
print(set0.symmetric_difference(set1))
print(set0.symmetric_difference(set2))
print(set1.symmetric_difference(set2))

NameError: name 'set0' is not defined

In [62]:

# adjacency_data.index = pd.CategoricalIndex(adjacency_data, categories=cat.categories)

sub2 = pd.Categorical(adjacency_data.index)
sub3 = pd.Categorical(adjacency_data.columns)
display(pd.DataFrame([sub.categories,sub2.categories,sub3.categories]).T)
Adj = adjacency_data.values
Adj

,0,1,2
0,agriculture,agriculture,agriculture
1,"animal science, poultry science","animal science, poultry science","animal science, poultry science"
2,biology,biology,biology
3,biomedical sciences,biomedical sciences,biomedical sciences
4,chemistry,chemistry,chemistry
...,...,...,...
34,political science,political science,political science
35,psychology,psychology,psychology
36,social work,social work,social work
37,sociology,sociology,sociology


array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 0, 1],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 1]], dtype=int64)

In [1]:
from setup import *
from scipy.special import binom
judge_data = pd.DataFrame.from_csv('judges.csv')
project_data = pd.DataFrame.from_csv('projects.csv')

def make_df(data):
    df = pd.DataFrame()    
    df['name'] = data['LAST NAME'] + ', ' + data['FIRST NAME']
    df['univ'] = data['INSTITUTION']
    df['cat'] = data['CATEGORY']
    df['subfield'] = data['SUBFIELD']
    df = df.apply(lambda x: x.str.lower() if x.dtype=='O' else x)
    return df

judges = make_df(judge_data)
judges['mobile'] = judge_data['MOBILE COMFORT LEVEL']
projects = make_df(project_data)
projects = projects[projects['subfield'] != 'doctoral']

def uniq(col):
    ser = pd.concat([judges[col],projects[col]])
    vc = ser.value_counts()
    idx, name = pd.factorize(ser)
    return ct, sorted(ct.index.tolist())

univ_ct, univs = uniq('univ')
cat_ct, cats = uniq('cat')
subfield_ct, subfields = uniq('subfield')

In [57]:
proj_sub.categories

Index(['agriculture', 'animal science, poultry science', 'biology',
       'biomedical sciences', 'chemistry', 'communication, media',
       'computer science, engineering, systems', 'counseling',
       'criminal justice, criminology, forensics', 'education',
       'education - special ed', 'engineering - civil',
       'engineering - electrical',
       'engineering - environmental, biological, agricultural',
       'engineering - mechanical', 'engineering - other',
       'english, literature', 'entomology',
       'environmental studies - plant sciences, forestry',
       'environmental studies - soil sciences', 'geography, gis',
       'geosciences, geology, hydrology, geophysics', 'health - public, human',
       'history', 'human resource management, development',
       'kinesiology, sport sciences', 'language, linguistics', 'management',
       'management information systems', 'marketing', 'math & statistics',
       'nursing', 'organizational behavior', 'physics', 'politic

In [71]:
### Compute field distance matrix
def show():
    print()
    print("k=%d"%k)
    print("P")
    display(P,'all')
    print("Dist")
    display(Dist,'all')
    
s = Adj.sum(axis=1)
P = Adj/(s[:,np.newaxis])

Dist = Adj.copy().astype(int)
k = 1
# print("Adj")
# print(Adj)
# show()
for k in range(2,50):
    # Multiply S by itself
    P = P.dot(P)
    # Write k in all entries of Dist that are 0 and the corresponding entry of A just became non-zero
    Dist[((Dist==0) & (P>0))] = k
    if(Dist.min() > 0): #break once Dist is full
        break
#     show()
# show()
Dist[Dist==0] = 999
Dist[np.diag_indices_from(Dist)] = 0
display(Dist,'all')

field_incompat = np.array([[Dist[jf,pf] for pf in proj_field.codes] for jf in judge_field.codes])
univ_match = np.array([[ju == pu for pu in proj_univ] for ju in judge_univ])
field_incompat[univ_match] = 999
display(field_incompat)
display(univ_match)


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0,1,2,3,2,2,3,3,3,1,...,3,3,3,3,2,3,2,2,3,2
1,1,0,1,2,2,3,3,3,3,2,...,3,3,3,4,2,3,3,3,3,1
2,1,1,0,1,2,3,3,3,3,2,...,3,3,2,3,2,3,3,3,3,1
3,2,2,1,0,1,2,3,2,2,2,...,3,3,1,3,3,2,2,2,2,2
4,2,2,1,1,0,3,3,3,3,3,...,3,3,2,3,2,3,3,3,3,2
5,4,4,4,3,4,0,3,3,3,2,...,1,4,4,2,4,2,2,3,3,4
6,3,3,3,2,2,2,0,3,3,2,...,2,1,3,2,2,2,2,3,3,3
7,5,5,4,4,4,3,4,0,1,2,...,3,4,4,4,4,2,1,1,2,5
8,5,5,4,4,4,3,4,1,0,2,...,3,4,4,4,4,1,1,1,1,5
9,4,4,4,4,4,1,4,2,2,0,...,2,4,4,3,4,2,1,1,2,4


In [25]:
set1 = set(adjacency_data.index)
set2 = set(adjacency_data.columns)
print(set0.symmetric_difference(set1))
print(set0.symmetric_difference(set2))
print(set1.symmetric_difference(set2))

w = [subfields,adjacency_data.index.tolist(),adjacency_data.columns.tolist()]
display(pd.DataFrame(w).T)
# display(adjacency_data.values)

set()
set()
set()


,0,1,2
0,agriculture,agriculture,agriculture
1,"animal science, poultry science","animal science, poultry science","animal science, poultry science"
2,biology,biology,biology
3,biomedical sciences,biomedical sciences,biomedical sciences
4,chemistry,chemistry,chemistry
...,...,...,...
34,political science,political science,political science
35,psychology,psychology,psychology
36,social work,social work,social work
37,sociology,sociology,sociology


In [9]:
l = subfields
d = {v: k for k, v in enumerate(l)}
d


{'agriculture': 0,
 'animal science, poultry science': 1,
 'biology': 2,
 'biomedical sciences': 3,
 'chemistry': 4,
 'communication, media': 5,
 'computer science, engineering, systems': 6,
 'counseling': 7,
 'criminal justice, criminology, forensics': 8,
 'education': 9,
 'education - special ed': 10,
 'engineering - civil': 11,
 'engineering - electrical': 12,
 'engineering - environmental, biological, agricultural': 13,
 'engineering - mechanical': 14,
 'engineering - other': 15,
 'english, literature': 16,
 'entomology': 17,
 'environmental studies - plant sciences, forestry': 18,
 'environmental studies - soil sciences': 19,
 'geography, gis': 20,
 'geosciences, geology, hydrology, geophysics': 21,
 'health - public, human': 22,
 'history': 23,
 'human resource management, development': 24,
 'kinesiology, sport sciences': 25,
 'language, linguistics': 26,
 'management': 27,
 'management information systems': 28,
 'marketing': 29,
 'math & statistics': 30,
 'nursing': 31,
 'organi

In [66]:
display(Dist,'all')

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,0,1,2,3,2,2,3,3,3,1,...,3,3,3,3,2,3,2,2,3,2
1,1,0,1,2,2,3,3,3,3,2,...,3,3,3,4,2,3,3,3,3,1
2,1,1,0,1,2,3,3,3,3,2,...,3,3,2,3,2,3,3,3,3,1
3,2,2,1,0,1,2,3,2,2,2,...,3,3,1,3,3,2,2,2,2,2
4,2,2,1,1,0,3,3,3,3,3,...,3,3,2,3,2,3,3,3,3,2
5,4,4,4,3,4,0,3,3,3,2,...,1,4,4,2,4,2,2,3,3,4
6,3,3,3,2,2,2,0,3,3,2,...,2,1,3,2,2,2,2,3,3,3
7,5,5,4,4,4,3,4,0,1,2,...,3,4,4,4,4,2,1,1,2,5
8,5,5,4,4,4,3,4,1,0,2,...,3,4,4,4,4,1,1,1,1,5
9,4,4,4,4,4,1,4,2,2,0,...,2,4,4,3,4,2,1,1,2,4


In [73]:
judge_field

[education, geosciences, geology, hydrology, geophysics, computer science, engineering, systems, environmental studies - plant sciences, forestry, psychology, ..., biology, human resource management, development, psychology, kinesiology, sport sciences, management information systems]
Length: 94
Categories (39, object): [agriculture, animal science, poultry science, biology, biomedical sciences, ..., psychology, social work, sociology, wildlife & fisheries management science]

In [81]:
# Make field_incompatibility matrix num_judg x num_post, filled with Dist between judge field and poster field
# field_incompat = np.fromfunction(lambda j,p: Dist[judg_field[j], post_field[p]], (num_judg,num_post), dtype=int)
num_judge = len(judge_field)
num_proj = len(proj_field)
print(num_judge)
print(num_proj)
print(judge_field.codes)

94
247
[ 9 21  6 18 35 13 18  4  2 35  2 17  4 35 24 25  6 28 25  7 13  3  8 25 27
 31 35  2  8 38 34 31  6  1  0 33  1 13  1  6 18 11  3 19 30 33  5 29 36  9
 32 30 35 16 30  9  9 35  8 20 23  7  9 35 18  0 13 22 15  0 38  3 30  9 35
  0 22 25 12 35  2  1  2 17 23  2  2 16 12  2 24 35 25 28]


In [95]:
field_incompat = np.array([[Dist[jf,pf] for pf in proj_field.codes] for jf in judge_field.codes])
univ_match = np.array([[ju == pu for pu in proj_univ] for ju in judge_univ])
field_incompat[univ_match] = 999
display(field_incompat)
display(univ_match)

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
0,999,999,999,999,999,999,999,999,999,999,...,4,4,4,4,4,4,4,4,4,4
1,999,999,999,999,999,999,999,999,999,999,...,1,1,1,2,2,3,1,2,1,3
2,999,999,999,999,999,999,999,999,999,999,...,2,2,2,2,2,3,2,3,2,3
3,999,999,999,999,999,999,999,999,999,999,...,2,2,2,3,3,3,2,1,2,3
4,999,999,999,999,999,999,999,999,999,999,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2,1,1,1,1,1,1,1,1,1,...,2,2,2,3,3,2,2,0,2,3
90,1,3,3,3,3,3,3,3,3,3,...,3,3,3,4,4,3,3,3,3,2
91,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
92,1,3,3,3,3,3,3,3,3,3,...,4,4,3,4,4,1,4,3,3,999


,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,246
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
90,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
91,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
92,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [86]:
np.fromfunction(lambda j,p: j, (judge_field.codes,proj_field.codes), dtype=int)

# Prevent judge and poster from same univ.  Entry is True is from same univ
univ_match = np.fromfunction(lambda j,p: judg_univ[j] == post_univ[p], (num_judg,num_post), dtype=int)

# Write 999 in corresponding entry of field_incompat
field_incompat[univ_match] = 999
print(field_incompat)

TypeError: only integer scalar arrays can be converted to a scalar index

In [15]:
from setup import *
from scipy.special import binom

#This is the algorithm for a SINGLE major category.  We will repeat for each category independently.

num_trials = 10 # number of judge->poster assignments analyzed for EACH poster->session assignment
cover = 2 # number of judgings per poster
num_post = 7 # number of posters in this category
num_judg = 5 # number of judges in this category
num_field = 5 # total numder of subfields in ALL categories.  We do not need to split subfields into categories - algorithm handles)
num_ses = 4 #number of sessions
num_univ = 8 #number of universities participating - must enumerate

#########################################################
# Code here picks random values to stand in for real data we will have after registration completes

#Randomly assign judge and poster fields
judg_field = np.random.randint(num_field, size=num_judg)
judg_univ = np.random.randint(num_univ, size=num_judg)
post_field = np.random.randint(num_field, size=num_post)
post_univ = np.random.randint(num_univ, size=num_post)

#Make random field adjency matrix
R = np.random.rand(num_field, num_field)
R = np.triu(R, k=1)
R = (R + R.T)  #must be symmetric
# print(R)
Adj = np.zeros_like(R).astype(int)
Adj[R>.7] = 1
s = Adj.sum(axis=1)
print(s)
P = Adj/(s[:,np.newaxis])
print(P)

#########################################################
# Real code starts here

### Compute field distance matrix
def show():
    print()
    print("k=%d"%k)
    print("P")
    print(P)
    print("Dist")
    print(Dist)
    

Dist = Adj.copy().astype(int)
k = 1
print("Adj")
print(Adj)
show()
for k in range(2,50):
    # Multiply S by itself
    P = P.dot(P)
    # Write k in all entries of Dist that are 0 and the corresponding entry of A just became non-zero
    Dist[((Dist==0) & (P>0))] = k
    if(Dist.min() > 0): #break once Dist is full
        break
    show()
show()
Dist[Dist==0] = 999
Dist[np.diag_indices_from(Dist)] = 0
print(Dist)

# Make field_incompatibility matrix num_judg x num_post, filled with Dist between judge field and poster field
field_incompat = np.fromfunction(lambda j,p: Dist[judg_field[j], post_field[p]], (num_judg,num_post), dtype=int)

# Prevent judge and poster from same univ.  Entry is True is from same univ
univ_match = np.fromfunction(lambda j,p: judg_univ[j] == post_univ[p], (num_judg,num_post), dtype=int)

# Write 999 in corresponding entry of field_incompat
field_incompat[univ_match] = 999
print(field_incompat)


#########################################################
# Compute judge workload per round
q = int(np.floor(num_post/num_ses)) #min number of posters per session
num_post_ses = q*np.ones(num_ses).astype(int) #make vector of length num_ses filled with q
r = num_post - q*num_ses #remainder, if num_ses does not divide num_post evenly
num_post_ses[:r] += 1 #spread these remainders across sessions
num_post_ses = num_post_ses[::-1] #reverse so that first session has fewest (for latecommers)
print(num_post_ses)
workload = np.ceil(num_post_ses.max()*cover/num_judg)
print("Judge workload = %d"%workload)


#########################################################
# Make poster->session assignment generator

def make_all_post_to_ses():
    """
    Generate all possible ways to assign posters to sessions.  Two steps:
    1. Select num_ses posters in increasing order to be the first posters assigned to each ses.
    2. Fill the remaining slots in each ses
    
    Explanation - Step 1 prevents redundancy seen in earlier, more naive versions of this algorithm.
    Example - Suppose num_ses = 2 and num_post = 4.  We do not want to see both
    post_ses = [[0,1],[2,3]] and post_ses = [[2,3],[0,1]].  These are redundant.  
    If you flip ses 0 and ses 1, it is the same (except posters 2&3 present in the earlier session).    
    """

    N = num_post # tracks number of posters not yet assigned to a session
    k = num_ses
    first_post_idx_gen = it.combinations(range(N), k) #generates all ways to select first for each ses in increading order
    N -= k # there are now num_ses fewer poster left to assign
    other_post_idx = []
    for ses in range(num_ses):
        k = num_post_ses[ses]-1 # Number of slots still open in this ses.  Recall - there's already 1 assigned.
        other_post_idx.append(it.combinations(range(N), k)) #generates all ways to fill open slots
        N -= k # fewer posters left to assign
    other_post_idx_gen = it.product(*other_post_idx) #combines the separate generators for each session into one big generator
    return first_post_idx_gen, other_post_idx_gen

#########################################################
# The code below is just a consistency check of the code above.  It computes the total number
# of ways to assign posters to sessions in two distinct ways.  Should produce same answer.
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
s = 0
for _ in first_post_idx_gen:
    s += 1
t = 0
for _ in other_post_idx_gen:
    t += 1
print(s*t)

# count a different way to verify
cs = np.array(num_post_ses) - 1
cs = cs.cumsum().tolist()
cs.insert(0,0)
N = num_post
u = binom(N,num_ses)
N -= num_ses
v = [binom(N-cs[i],num_post_ses[i]-1) for i in range(num_ses)]
print(int(u*np.prod(v)))

#########################################################
#Assign judges to posters
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
best_post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
best_judg_post = [[]]*num_ses
best_fi_score_tot = 999999

for first_post_idx in first_post_idx_gen:
    
    for other_post_idx in other_post_idx_gen:
        
        L = list(range(num_post))
        post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
        
        # We will walk first_post_idx and other_post_idx backward because we will use the pop method to remove
        # used posters from L.  When an element is popped from L, all elements to its right slide
        # one slot to the left.  But all elements to its left do not move.  Thus, we want to pop from
        # right to left so indexing is consistent.  Example, let L=[0,1,2,3].  let post_idx = [0,2]
        # Thus, we expect to pop 0 and 2 from L.  Suppose we pop 0 first.  Now L = [1,2,3].  When we
        # pop 2, we actually get 3, because 3 is now in slot 2.  However, if we pop 2 first,
        # L = [0,1,2].  Now we pop 0 and get 0.  This is the desired behavior.
        # Recall [::-1] reverses a list        
        for (ses, post_idx) in enumerate(first_post_idx[::-1]):
            # we use -(ses+1) so that the smallest index goes into the first ses and the largest goes into the last ses.
            # We also put into the last slot of each ses (not first slot as discussed above).
            # This is merely a cosmetic preference - makes posters within the same ses appear
            # in a consistent order. But the order of posters within the same ses does not actually matter.
            post_ses[-(ses+1)][-1] = L[post_idx]
            L.pop(post_idx)

        for (ses, post_idx) in enumerate(other_post_idx):
            for (i, j) in enumerate(post_idx[::-1]): # Again, reverse so pop works right (see above)
                post_ses[ses][i] = L.pop(j)
            post_ses[ses] = post_ses[ses][::-1] # Again, cosmetic.  The posters in each session
# are listed in descending order.  This reverses into ascending order.  Turn off to increase speed.
#         print(post_ses)    
    
        #########################################################
        # Assign judges to posters
        best_fi_ses = [[]] * num_ses #best fi matrix
        best_fi_score_ses = [999] * num_ses #best fi_score
        print()
        print("Optimizing poster->session %s"%post_ses)

        for (ses, post) in enumerate(post_ses):  
            print("Optimizing session %d with posters %s"%(ses,post))
            workload = np.ceil(len(post)*cover/num_judg)
#             print(workload)
            for trial in range(num_trials):
                skip_trial = False
                field_incompat_sub = field_incompat[:,post] #Extract cols for posters assigned to this ses
#                 print(field_incompat_sub)
                fi = np.zeros_like(field_incompat_sub)
                judg_work = [0]*num_judg #Tracks each judge's workload
                lot_order = np.random.permutation(len(post)) #randomize poster lottery order
                for i in range(cover): # number of "rounds" of the lottery = cover
                    if(skip_trial == True):
                        break
#                     print("round = %d"%i)
                    for col in lot_order:
#                         print("col = %d"%col)
                        c = field_incompat_sub[:,col] #Grab column for this poster
                        m = c.min()
                        if(m == 999): #If no legal judge remains, abandon this trial
                            skip_trial = True
                            break
                        rows = np.where(c == m)[0] #find all rows in this col where the smallest fi appears
#                         print(rows)
                        if(len(rows) == 1):
                            row = rows[0]
                        else:
                            row = np.random.choice(rows) #choose one such row at random
#                         print(row)
                        fi[row,col] = field_incompat_sub[row,col] #record fi for this pairing
                        judg_work[row] += 1 #increment that judge's workload tracker
                        if(judg_work[row] >= workload):
                            field_incompat_sub[row,:] = 999 #if judge has reached workload limit, fill her row with 999
                        else:
                            field_incompat_sub[row,col] = 999 #else, put 999 in the (row,col) entry only
#                         print(fi)
#                         print(field_incompat_sub)
                if(skip_trial == True):
                    continue
                fi_score = fi.sum() #total fi score for this judge->poster pairing for this ses
#                 print(field_incompat_sub)
#                 print(fi>0)
#                 print(fi)
#                 print(fi_score)
                if(fi_score < best_fi_score_ses[ses]): # if current beats prior best, keep it
                    print("current ses %d fi = %d is better than prior best ses %d fi = %d.  I'll keep it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
                    best_fi_ses[ses] = fi.copy()
                    best_fi_score_ses[ses] = fi_score
                else:
                    print("current ses %d fi = %d is not better than prior best ses %d fi = %d.  I'll ignore it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
#                 print(best_fi_score_ses[ses])
                if(best_fi_score_ses[ses] <= 1):
                    print("Found best possible j->p for ses %d.  Moving on."%ses)        
        print("Finished optimizing session %d"%ses)
        #########################################################
        # We now have the best way to assign judges0>posters under the current poster->session assignment
        # We check if it beats the best from prior poster->session assignment
        print("Finished optimizing poster->session %s"%post_ses)
        fi_score_tot = sum(best_fi_score_ses)
        if(fi_score_tot < best_fi_score_tot):
            print("current total fi = %d is better than prior best total fi = %d.  I'll keep it."%(fi_score_tot,best_fi_score_tot))
            best_post_ses = post_ses.copy()
            best_judg_post = best_fi_ses.copy()
            best_fi_score_tot = fi_score_tot            
            for (ses,(ps,jp)) in enumerate(zip(best_post_ses,best_judg_post)):
                df = pd.DataFrame(jp>0, columns=ps)
                print("session %d"%ses)
                display(df)
                print()
        else:
            print("current total fi = %d is not better than prior best total fi = %d.  I'll ignore it."%(fi_score_tot,best_fi_score_tot))
                

[[ 0.          0.95172283  0.50505158  0.46856326  0.71351086]
 [ 0.          0.          0.48358181  0.97739826  0.67615936]
 [ 0.          0.          0.          0.90121639  0.05429828]
 [ 0.          0.          0.          0.          0.9633711 ]
 [ 0.          0.          0.          0.          0.        ]]
[2 2 1 3 2]
[[ 0.          0.5         0.          0.          0.5       ]
 [ 0.5         0.          0.          0.5         0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [ 0.          0.33333333  0.33333333  0.          0.33333333]
 [ 0.5         0.          0.          0.5         0.        ]]
Adj
[[0 1 0 0 1]
 [1 0 0 1 0]
 [0 0 0 1 0]
 [0 1 1 0 1]
 [1 0 0 1 0]]

k=1
P
[[ 0.          0.5         0.          0.          0.5       ]
 [ 0.5         0.          0.          0.5         0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [ 0.          0.33333333  0.33333333  0.          0.33333333]
 [ 0.5         0.          